In [1]:
import s3fs 
import boto3
import zipfile
import os


In [2]:
!aws s3 ls eco-w1/in0/mrlc_zip/

2020-12-26 14:25:49 10086708780 Annual_Herb_1985_1995.zip
2020-12-26 14:28:47 10211619774 Annual_Herb_1996_2006.zip
2020-12-26 14:31:47 10245707594 Annual_Herb_2007_2018.zip
2020-12-26 14:34:44 25895252470 Herb_1985_1995.zip
2020-12-26 14:42:11 25858781544 Herb_1996_2006.zip
2020-12-26 14:50:07 20779156209 Herb_2007_2018.zip


In [3]:
s3 = s3fs.S3FileSystem()
bucket_path='eco-w1/in0/'
s3.ls(bucket_path)

['eco-w1/in0/cflux', 'eco-w1/in0/mrlc_zip', 'eco-w1/in0/rapv2']

In [4]:
bucket_path='eco-w1/in0/mrlc_zip'

mrlc_files=s3.ls(bucket_path)

In [5]:
for file_obj in mrlc_files:
    if 'Annual' in file_obj:
        print(file_obj)

eco-w1/in0/mrlc_zip/Annual_Herb_1985_1995.zip
eco-w1/in0/mrlc_zip/Annual_Herb_1996_2006.zip
eco-w1/in0/mrlc_zip/Annual_Herb_2007_2018.zip


In [6]:

def download_s3obj_to_local_file(s3_full_path, local_dir):
    bucket_name = s3_full_path.split('/')[0]
    object_name = '/'.join(s3_full_path.split('/')[1:])
    local_full_path = local_dir + '/' + s3_full_path.split('/')[-1]
    
    s3 = boto3.client('s3')
    s3.download_file(bucket_name, object_name, local_full_path)

In [7]:
def unzip_me(path_to_zip_file, directory_to_extract_to):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

In [8]:
#!rm -fr mrlc_ebs_data/

In [9]:
def unzip_file_obj(s3_full_path, directory_to_extract_to):
        local_full_path = local_dir + '/' + s3_full_path.split('/')[-1]
        unzip_me(local_full_path, directory_to_extract_to)
        print('removing file: ',local_full_path)
        os.remove(local_full_path)

In [10]:
def _mkdir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

In [11]:
def gdal_convert_img_tif(input_img, output_tif):
    cmd = f'gdal_translate -of GTiff {input_img} {output_tif}'
    print(cmd)
    os.system(cmd)

In [30]:
local_dir = 'mrlc_ebs_data'
directory_to_extract_to = 'mrlc_ebs_data/mrlc_erdas_img'
tif_output_dir = 'mrlc_ebs_data/mrlc_tif'
cog_output_dir = 'mrlc_ebs_data/mrlc_cog'


In [32]:
def convert_erdas_img_to_tif(tif_output_dir):
    arr = os.listdir(directory_to_extract_to)
    subdir=arr[0]
    files = os.listdir(f'{directory_to_extract_to}/{subdir}')
    for file in files:
        if file.endswith('.img'):
            full_file = f'{directory_to_extract_to}/{subdir}/{file}'
            print(full_file)
            input_img = full_file
            tif_file=file.replace('.img', '.tif')
            output_tif=f'{tif_output_dir}/{tif_file}'
            gdal_convert_img_tif(input_img, output_tif)
    os.remove(f'{directory_to_extract_to}/{subdir}')

In [45]:
from loganLib.cog_func import cog_create_from_tif
help(cog_create_from_tif)

Help on function cog_create_from_tif in module loganLib.cog_func:

cog_create_from_tif(src_tif, dst_cog)



In [46]:
def convert_tif_cog_single(tif_input, cog_output):
    print(tif_input,cog_output)
    cog_create_from_tif(tif_input,cog_output)

In [47]:
def convert_tif_cog(tif_output_dir, cog_output_dir):
    files = os.listdir(tif_output_dir)
    for file in files:
        if(file.endswith('.tif')):
            print(file)
            tif_input = f'{tif_output_dir}/{file}'
            cog_output = f'{cog_output_dir}/{file}'
            convert_tif_cog_single(tif_input, cog_output)
            os.remove(tif_input)
    

In [ ]:
_mkdir(local_dir)
_mkdir(directory_to_extract_to)
_mkdir(tif_output_dir)
_mkdir(cog_output_dir)

for file_obj in mrlc_files[0:1]:
    if 'Annual' in file_obj:
        print(file_obj)
        #download_s3obj_to_local_file(file_obj, local_dir)
        #unzip_file_obj(file_obj,directory_to_extract_to)
        #convert_erdas_img_to_tif(tif_output_dir)
        convert_tif_cog(tif_output_dir, cog_output_dir)
        

eco-w1/in0/mrlc_zip/Annual_Herb_1985_1995.zip
nlcd_anhb_1986_mos_v1.tif
mrlc_ebs_data/mrlc_tif/nlcd_anhb_1986_mos_v1.tif mrlc_ebs_data/mrlc_cog/nlcd_anhb_1986_mos_v1.tif
nlcd_anhb_1985_mos_v1.tif
mrlc_ebs_data/mrlc_tif/nlcd_anhb_1985_mos_v1.tif mrlc_ebs_data/mrlc_cog/nlcd_anhb_1985_mos_v1.tif
nlcd_anhb_1988_mos_v1.tif
mrlc_ebs_data/mrlc_tif/nlcd_anhb_1988_mos_v1.tif mrlc_ebs_data/mrlc_cog/nlcd_anhb_1988_mos_v1.tif
nlcd_anhb_1990_mos_v1.tif
mrlc_ebs_data/mrlc_tif/nlcd_anhb_1990_mos_v1.tif mrlc_ebs_data/mrlc_cog/nlcd_anhb_1990_mos_v1.tif


In [23]:
!ls -lh 'mrlc_ebs_data/mrlc_tif'

total 64G
-rw-r--r-- 1 jupyter-butzer jupyter-butzer 5.9G Jan 31 16:59 nlcd_anhb_1985_mos_v1.tif
-rw-r--r-- 1 jupyter-butzer jupyter-butzer  33K Jan 31 16:59 nlcd_anhb_1985_mos_v1.tif.aux.xml
-rw-r--r-- 1 jupyter-butzer jupyter-butzer 5.9G Jan 31 17:01 nlcd_anhb_1986_mos_v1.tif
-rw-r--r-- 1 jupyter-butzer jupyter-butzer  33K Jan 31 17:01 nlcd_anhb_1986_mos_v1.tif.aux.xml
-rw-r--r-- 1 jupyter-butzer jupyter-butzer 5.9G Jan 31 16:57 nlcd_anhb_1987_mos_v1.tif
-rw-r--r-- 1 jupyter-butzer jupyter-butzer  33K Jan 31 16:57 nlcd_anhb_1987_mos_v1.tif.aux.xml
-rw-r--r-- 1 jupyter-butzer jupyter-butzer 5.9G Jan 31 17:04 nlcd_anhb_1988_mos_v1.tif
-rw-r--r-- 1 jupyter-butzer jupyter-butzer  33K Jan 31 17:04 nlcd_anhb_1988_mos_v1.tif.aux.xml
-rw-r--r-- 1 jupyter-butzer jupyter-butzer 5.9G Jan 31 16:55 nlcd_anhb_1989_mos_v1.tif
-rw-r--r-- 1 jupyter-butzer jupyter-butzer  33K Jan 31 16:55 nlcd_anhb_1989_mos_v1.tif.aux.xml
-rw-r--r-- 1 jupyter-butzer jupyter-butzer 5.9G Jan 31 17:00 nlcd_anhb_1990_mos_

In [24]:
! df -h .

Filesystem      Size  Used Avail Use% Mounted on
/dev/nvme0n1p1  427G  418G  9.0G  98% /


In [16]:
arr = os.listdir(local_dir)
arr

['mrlc_tif', 'Annual_Herb_1985_1995.zip', 'mrlc_erdas_img']

In [17]:
arr = os.listdir(directory_to_extract_to)
files = os.listdir(f'{directory_to_extract_to}/{arr[0]}')
for file in files:
    if file.endswith('.img'):
        print(file)

nlcd_anhb_1992_mos_v1.img
nlcd_anhb_1989_mos_v1.img
nlcd_anhb_1993_mos_v1.img
nlcd_anhb_1987_mos_v1.img
nlcd_anhb_1995_mos_v1.img
nlcd_anhb_1985_mos_v1.img
nlcd_anhb_1990_mos_v1.img
nlcd_anhb_1986_mos_v1.img
nlcd_anhb_1994_mos_v1.img
nlcd_anhb_1991_mos_v1.img
nlcd_anhb_1988_mos_v1.img


In [18]:
! df -h .
!ls -lh mrlc_ebs_data/mrlc_erdas_img

Filesystem      Size  Used Avail Use% Mounted on
/dev/nvme0n1p1  427G  418G  9.0G  98% /
total 4.0K
drwxr-xr-x 2 jupyter-butzer jupyter-butzer 4.0K Jan 30 21:37 Annual_Herb_1985_1995


In [19]:
#!rm -fr mrlc_ebs_data/

In [26]:
! df -h .

Filesystem      Size  Used Avail Use% Mounted on
/dev/nvme0n1p1  427G  310G  117G  73% /


In [21]:
! gdal_translate -of GTiff mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_1985_1995/nlcd_anhb_1992_mos_v1.img mrlc_ebs_data/mrlc_tif/nlcd_anhb_1992_mos_v1.tif

Input file size is 71204, 87666
0...10...20...30...40...50...60...70...80...90...100 - done.


In [25]:
! rm -fr mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_1985_1995/